**SFT (Supervised Fine-Tuning, instruction-tuning)**  — метод адаптации предварительно обученных языковых моделей (LLM) под конкретную задачу с помощью размеченных данных

**Цель** — скорректировать веса модели так, чтобы она лучше справлялась с задачей, не теряя при этом общие знания, полученные в ходе предварительного обучения

### Установка зависимостей

In [1]:
!pip install transformers accelerate
!huggingface-cli login
!pip install --upgrade pip
!pip install --force-reinstall "unsloth[cu121-torch240] @ git+https://github.com/unslothai/unsloth.git"
!pip install --force-reinstall unsloth_zoo

⚠️  Warning: 'huggingface-cli login' is deprecated. Use 'hf auth login' instead.

    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) т
Invalid input. Must be one of ('y', 'yes', '1', 'n', 'no', '0', '')
Add token as git credential? (Y/n) n
Token is valid (permission: fineGrained).
The token

  Using cached unsloth_zoo-2025.9.14-py3-none-any.whl.metadata (31 kB)
  Using cached torch-2.8.0-cp312-cp312-manylinux_2_28_x86_64.whl.metadata (30 kB)
  Using cached torchao-0.13.0-1-cp39-abi3-manylinux_2_24_x86_64.manylinux_2_28_x86_64.whl.metadata (19 kB)
  Using cached triton-3.4.0-cp312-cp312-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl.metadata (1.7 kB)
  Using cached packaging-25.0-py3-none-any.whl.metadata (3.3 kB)
  Using cached tyro-0.9.32-py3-none-any.whl.metadata (11 kB)
  Using cached transformers-4.56.2-py3-none-any.whl.metadata (40 kB)
  Using cached datasets-4.1.1-py3-none-any.whl.metadata (18 kB)
  Using cached sentencepiece-0.2.1-cp312-cp312-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl.metadata (10 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached psutil-7.1.0-cp36-abi3-manylinux_2_12_x86_64.manylinux2010_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (23 kB)
  Using cached wheel-0.45.1-py3-none-any.whl.metadata (2.

In [ ]:
# !pip install --upgrade pip
# !pip install --force-reinstall "unsloth[cu121-torch240] @ git+https://github.com/unslothai/unsloth.git"
# !pip install --force-reinstall unsloth_zoo


### Model SQLCODER-7b

In [1]:
import unsloth
from unsloth import FastLanguageModel
from transformers import TextStreamer
import torch

MODEL_NAME = "defog/sqlcoder-7b"

# загрузка модели
model_sql, tokenizer = FastLanguageModel.from_pretrained(
    model_name=MODEL_NAME,
    max_seq_length=2048,
    load_in_4bit=True
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


    PyTorch 2.4.1+cu121 with CUDA 1201 (you have 2.8.0+cu128)
    Python  3.12.6 (you have 3.12.11)
  Please reinstall xformers (see https://github.com/facebookresearch/xformers#installing-xformers)
  Memory-efficient attention, SwiGLU, sparse and more won't be available.
  Set XFORMERS_MORE_DETAILS=1 for more details


Switching to PyTorch attention since your Xformers is broken.

Unsloth: Xformers was not installed correctly.
Please install xformers separately first.
Then confirm if it's correctly installed by running:
python -m xformers.info

Longer error message:
xFormers can't load C++/CUDA extensions. xFormers was built for:
    PyTorch 2.4.1+cu121 with CUDA 1201 (you have 2.8.0+cu128)
    Python  3.12.6 (you have 3.12.11)
  Please reinstall xformers (see https://github.com/facebookresearch/xformers#installing-xformers)
  Memory-efficient attention, SwiGLU, sparse and more won't be available.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.9.11: Fast Mistral patching. Transformers: 4.56.2.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu128. CUDA: 7.5. CUDA Toolkit: 12.8. Triton: 3.4.0
\        /    Bfloat16 = FALSE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.co

pytorch_model.bin.index.json: 0.00B [00:00, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

...

tokenizer_config.json:   0%|          | 0.00/915 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

defog/sqlcoder-7b does not have a padding token! Will use pad_token = <unk>.


### Fine Tuning

#### Dataset

In [2]:
from datasets import load_dataset

dataset = load_dataset("json", data_files="queries_marked_602_915.jsonl")
print(dataset)

full_dataset = dataset["train"]


temp_split = full_dataset.train_test_split(test_size=0.2, seed=42, shuffle=True)
train_dataset = temp_split["train"]
temp = temp_split["test"]

val_test = temp.train_test_split(test_size=0.5, seed=42, shuffle=True)

eval_dataset = val_test["train"]
test_dataset = val_test["test"]
print(train_dataset[0])
print(len(train_dataset), len(eval_dataset), len(test_dataset))

Generating train split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'output'],
        num_rows: 1518
    })
})
{'instruction': 'Optimize the SQL query', 'input': "SELECT * FROM assets WHERE serial_number ~ '[0-9]{3}' AND serial_number IS NOT NULL;", 'output': "SELECT * FROM assets WHERE serial_number REGEXP '[0-9]{3}';"}
1214 152 152


#### Lora adaptor

Trial 18 finished with value: 0.341317355632782 and parameters:
- {'lora_r': 16, 'lora_alpha': 154, 'lora_dropout': 0.1873326662303232, 'use_rslora': True, 'target_modules': ['q_proj', 'k_proj', 'v_proj', 'o_proj', 'gate_proj', 'up_proj', 'down_proj'], 'learning_rate': 1.3887906641927588e-05, 'per_device_train_batch_size': 2, 'gradient_accumulation_steps': 8, 'weight_decay': 0.01652408175881535, 'num_train_epochs': 3, 'lr_scheduler_type': 'cosine_with_restarts', 'warmup_ratio': 0.08121051842939736, 'optim': 'adamw_bnb_8bit', 'fp16': False, 'bf16': True, 'gradient_checkpointing': False}. Best is trial 18 with value: 0.341317355632782.

In [3]:
model = FastLanguageModel.get_peft_model(
    model_sql,
    r = 16,
    lora_alpha = 154,
    lora_dropout = 0.1873326662303232,
    use_rslora = True,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    loftq_config = None,
)

Unsloth: Dropout = 0 is supported for fast patching. You are using dropout = 0.1873326662303232.
Unsloth will patch all other layers, except LoRA matrices, causing a performance hit.
Unsloth 2025.9.11 patched 32 layers with 0 QKV layers, 0 O layers and 0 MLP layers.


#### Setting up training parameters

In [4]:
from transformers import TrainingArguments
from trl import SFTTrainer
from unsloth import is_bfloat16_supported

# Параметры обучения
training_args = TrainingArguments(
    learning_rate=1.3887906641927588e-05,
    num_train_epochs=3, #num_train_epochs=2,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=8,
    optim="adamw_bnb_8bit",
    weight_decay=0.01652408175881535,
    lr_scheduler_type="cosine_with_restarts",
    fp16=True,
    bf16=False,
    gradient_checkpointing=False,
    logging_steps=10,
    eval_steps=25,
    save_steps=25,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    greater_is_better=False,
    metric_for_best_model="eval_loss", # Будем менять
    warmup_ratio=0.08121051842939736,
    seed=3407,
    output_dir="/content/fine-tuning/training_args",
    logging_dir="/content/fine-tuning/logs",
    report_to="none",
)

#### Creating and launching training

In [7]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

def tokenize_function(examples):
    tokenized_batch = {"input_ids": [], "attention_mask": [], "labels": []}

    for instr, inp, out in zip(examples["instruction"], examples["input"], examples["output"]):
        prompt = instr
        if inp and inp.strip():
            prompt += "\nInput: " + inp
        prompt += "\nOutput: " + out

        # Токенизация всего prompt
        tokenized = tokenizer(
            prompt,
            truncation=True,
            padding="max_length",
            max_length=512,
        )

        input_ids = tokenized["input_ids"]
        attention_mask = tokenized["attention_mask"]

        # Создаём labels: по умолчанию все токены игнорируем
        labels = [-100] * len(input_ids)

        # Находим позицию начала Output в prompt
        output_start = prompt.find("Output: ") + len("Output: ")
        # Токенизируем только Output
        output_ids = tokenizer(prompt[output_start:], truncation=True, max_length=512)["input_ids"]


        labels[-len(output_ids):] = output_ids

        tokenized_batch["input_ids"].append(input_ids)
        tokenized_batch["attention_mask"].append(attention_mask)
        tokenized_batch["labels"].append(labels)

    return tokenized_batch

tokenized_train = train_dataset.map(tokenize_function, batched=True)
tokenized_eval = eval_dataset.map(tokenize_function, batched=True)


Map:   0%|          | 0/1214 [00:00<?, ? examples/s]

Map:   0%|          | 0/152 [00:00<?, ? examples/s]

In [8]:
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_eval,
    max_seq_length=512,
    packing=False,
    args=training_args,
)

trainer.train()

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'pad_token_id': 2}.
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 1,214 | Num Epochs = 3 | Total steps = 228
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 8
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 8 x 1) = 16
 "-____-"     Trainable parameters = 41,943,040 of 7,283,675,136 (0.58% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Epoch,Training Loss,Validation Loss
1,1.502200,1.431508
2,0.984800,1.271242


Unsloth: Not an error, but MistralForCausalLM does not accept `num_items_in_batch`.
Using gradient accumulation will be very slightly less accurate.
Read more on gradient accumulation issues here: https://unsloth.ai/blog/gradient


Epoch,Training Loss,Validation Loss
1,1.502200,1.431508
2,0.984800,1.271242
3,0.983400,1.220239


TrainOutput(global_step=228, training_loss=1.3460788559495358, metrics={'train_runtime': 5644.0596, 'train_samples_per_second': 0.645, 'train_steps_per_second': 0.04, 'total_flos': 8.002492606867046e+16, 'train_loss': 1.3460788559495358, 'epoch': 3.0})

In [9]:
model.save_pretrained("/content/fine-tuning/Model/sqlcoder-finetuned_3")
tokenizer.save_pretrained("/content/fine-tuning/Model/sqlcoder-finetuned_3")

('/content/fine-tuning/Model/sqlcoder-finetuned_3/tokenizer_config.json',
 '/content/fine-tuning/Model/sqlcoder-finetuned_3/special_tokens_map.json',
 '/content/fine-tuning/Model/sqlcoder-finetuned_3/tokenizer.model',
 '/content/fine-tuning/Model/sqlcoder-finetuned_3/added_tokens.json',
 '/content/fine-tuning/Model/sqlcoder-finetuned_3/tokenizer.json')

In [10]:
!zip -r model_4.zip fine-tuning/Model

  adding: fine-tuning/Model/ (stored 0%)
  adding: fine-tuning/Model/sqlcoder-finetuned_3/ (stored 0%)
  adding: fine-tuning/Model/sqlcoder-finetuned_3/adapter_model.safetensors (deflated 7%)
  adding: fine-tuning/Model/sqlcoder-finetuned_3/tokenizer_config.json (deflated 68%)
  adding: fine-tuning/Model/sqlcoder-finetuned_3/README.md (deflated 65%)
  adding: fine-tuning/Model/sqlcoder-finetuned_3/adapter_config.json (deflated 56%)
  adding: fine-tuning/Model/sqlcoder-finetuned_3/tokenizer.json (deflated 85%)
  adding: fine-tuning/Model/sqlcoder-finetuned_3/special_tokens_map.json (deflated 73%)
  adding: fine-tuning/Model/sqlcoder-finetuned_3/tokenizer.model (deflated 55%)


In [11]:
from google.colab import files
files.download('model_4.zip')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from google.colab import files


test_dataset.to_json("test_dataset.jsonl", orient="records", lines=True)
files.download("test_dataset.jsonl")
